In [1]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.preprocessing import OneHotEncoder,StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

data = pd.read_csv('mumbai-house-price-data.csv')
data['property_type'] = data['property_type'].where(
    data['property_type'].str.contains('Apartment', case=False, na=False),
    None
)
data.drop(columns=['title','city','property_type','furnished','age','total_floors',	'latitude'	,'longitude','price_per_sqft'], inplace=True)

data['locality'] = data['locality'].apply(lambda x:x.strip())
locality_stats = data.groupby('locality')['locality'].agg('count').sort_values(ascending=False)
locality_stats_lt10 = locality_stats[locality_stats <= 20]
data['locality'] = data['locality'].apply(lambda x:'other' if x in locality_stats_lt10 else x)
data['sqft_per_bed'] = data['area']/data['bedroom_num']
data.dropna(inplace=True)
data['price']=round(data['price']/100000,2)
data2 = data[data['sqft_per_bed']>=200]
data2['price_per_sqft'] = round((data2['price'] *100000)/(data2['area']),2)
data3 = round(data2[data2['price_per_sqft']>=2000],2)
data3.drop(columns=['sqft_per_bed','price_per_sqft'], axis=1, inplace=True)
data3.dropna(inplace=True)

col_trans = make_column_transformer((OneHotEncoder(sparse_output=False),['locality']),remainder='passthrough')
lr= LinearRegression()
scaler =StandardScaler()
model = make_pipeline(col_trans,scaler,lr)
data_input = data3.drop(columns=['price'])
data_output = data3['price']
x_train, x_test, y_train, y_test = train_test_split(data_input,data_output,test_size=0.2)
model.fit(x_train,y_train)
model.score(x_test,y_test)

C:\Users\Umesh\AppData\Local\Temp\ipykernel_14124\2246274469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['price_per_sqft'] = round((data2['price'] *100000)/(data2['area']),2)


0.6498555217017963

In [2]:
input = pd.DataFrame([['600','Borivali','2','2','0']],columns=['area',	'locality',	'bedroom_num',	'bathroom_num','balcony_num'])
model.predict(input)

ValueError: 5 columns passed, passed data had 4 columns

In [ ]:
pk.dump(model,open('House_Prediction_Model.pkl','wb'))

data3.to_csv('Cleaned_Dataset.csv',)